In [1]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import random
from keras import backend as K
import cv2
from collections import Counter
from sklearn.utils import class_weight
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pickle
import tensorflow as tf
if not os.path.exists('pickled_3'):
    os.makedirs('pickled_3')

C:\Users\Pegasus\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print("loading images...")

def rotate_images(img):
    X_rotate = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (64, 64, 3))
    k = tf.placeholder(tf.int32)
    tf_img = tf.image.rot90(X, k = k)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(3):
            rotated_img = sess.run(tf_img, feed_dict = {X: img, k: i + 1})
            X_rotate.append(rotated_img)
            
    #X_rotate = np.array(X_rotate, dtype = np.float32)
    return X_rotate[0],X_rotate[1],X_rotate[2]


data = []
labels = []

if not os.path.exists('pickled_3/melanoma_data.pkl'):
        
    directory = os.fsencode("melanoma")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('melanoma\\' + filename)
        image = cv2.imread(path)
        
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        image_mask = cv2.imread(path)
        image = cv2.bitwise_and(image,image_mask)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        
        r1,r2,r3 = rotate_images(image)
        data.append(image)
        labels.append(1)
        data.append(r1)
        data.append(r2)
        data.append(r3)
        
        #imgplot = plt.imshow(image)
        #plt.show()
        
        labels.append(1)    
        labels.append(1)    
        labels.append(1)        
    
    directory = os.fsencode("others")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('others\\' + filename)
        image = cv2.imread(path)
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        image_mask = cv2.imread(path)
        image = cv2.bitwise_and(image,image_mask)
        
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        
        data.append(image)
        labels.append(0)
        
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickled_3/melanoma_data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickled_3/melanoma_data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']
                
print("Done.")

loading images...
Done.


In [3]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [4]:
batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64,64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('score = ',score[1])
y_p = model.predict(x_test)
print(classification_report(y_test, y_p.round()))

x_train shape: (2497, 64, 64, 3)
2497 train samples
625 test samples
Train on 2497 samples, validate on 625 samples
Epoch 1/10
2497/2497 [==============================] - 165s 66ms/step - loss: 0.6495 - acc: 0.6308 - val_loss: 0.6111 - val_acc: 0.6872
Epoch 2/10
2497/2497 [==============================] - 167s 67ms/step - loss: 0.5968 - acc: 0.6808 - val_loss: 0.5971 - val_acc: 0.6896
Epoch 3/10
2497/2497 [==============================] - 175s 70ms/step - loss: 0.5818 - acc: 0.6984 - val_loss: 0.5743 - val_acc: 0.7064
Epoch 4/10
2497/2497 [==============================] - 173s 69ms/step - loss: 0.5681 - acc: 0.7115 - val_loss: 0.5609 - val_acc: 0.7088
Epoch 5/10
2497/2497 [==============================] - 178s 71ms/step - loss: 0.5502 - acc: 0.7125 - val_loss: 0.5649 - val_acc: 0.6968
Epoch 6/10
2497/2497 [==============================] - 161s 64ms/step - loss: 0.5454 - acc: 0.7191 - val_loss: 0.5537 - val_acc: 0.6952
Epoch 7/10
2497/2497 [==============================] - 190s 7